In [7]:
import os

import keras
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [2]:
zoom_gen = ImageDataGenerator()
transform_parameters = {
    'zx': 0.6,
    'zy': 0.6,
}
zoom = lambda x: zoom_gen.apply_transform(x, transform_parameters)

In [3]:
train_datagen = ImageDataGenerator(
        rotation_range=40,
        fill_mode='nearest',
        preprocessing_function=zoom)
    
test_datagen = ImageDataGenerator(
        preprocessing_function=zoom)

In [5]:
batch_size = 32

train_generator = train_datagen.flow_from_directory(
        '../data/data/train',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale')

validation_generator = test_datagen.flow_from_directory(
        '../data/data/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale')

Found 1682 images belonging to 3 classes.
Found 472 images belonging to 3 classes.


In [29]:
test_generator = test_datagen.flow_from_directory(
        '../data/data/test',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale')

Found 519 images belonging to 3 classes.


In [25]:
if keras.backend.image_data_format() == 'channels_first':
    input_shape = (1, 150, 150)
else:
    input_shape = (150, 150, 1)
    
optimizer = optimizers.Nadam(lr=0.002,
              beta_1=0.9,
              beta_2=0.999,
              epsilon=1e-08,
              schedule_decay=0.004)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              #optimizer=optimizer,
              metrics=['accuracy'])

In [31]:
model.fit_generator(
    train_generator,
    steps_per_epoch=1682 // batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=472 // batch_size,
    workers=4,
    use_multiprocessing=True)
model.save_weights('first_try.h5')

Epoch 1/50
33/52 [==================>...........] - ETA: 8s - loss: 0.1193 - acc: 0.9678

Process ForkPoolWorker-16:
Process ForkPoolWorker-11:
Process ForkPoolWorker-9:
Process ForkPoolWorker-14:
Process ForkPoolWorker-12:
Process ForkPoolWorker-13:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process 

Epoch 1/50


Traceback (most recent call last):
Process ForkPoolWorker-15:
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
Traceback (most recent call last):
KeyboardInterrupt
  File "/usr/local/Cella

KeyboardInterrupt: 

In [30]:
model.evaluate_generator(test_generator, workers=4, use_multiprocessing=True)

[1.796463835675822, 0.7899807324069543]